In [1]:
from model import Posicao, Extrato, AwsModel
from views import FundoInvestimento, Acao, FundoImobiliario
from controllers import MainController
import numpy as np
import pandas as pd 
from datetime import datetime
import os

import plotly.graph_objs as go

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [2]:
main_controller = MainController()

WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (71838) not 512 + multiple of sector size (512)
WARNING *** file size (71834) not 512 + multiple of sector size (512)
WARNING *** file size (69788) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (60572) not 512 + multiple of sector size (512)
WARNING *** file size (59548) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59548) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59548) not 512 + multiple of sector size (512)
WARNING *** file siz

/Users/gabriellopes/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gabriellopes/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [3]:
df_tmp = main_controller.get_revenue_dataset()

In [4]:
df_tmp.head()

,Data,ano,mes,financeiro,aporte,retirada,rendimento,%,renda_acum,aporte_acum,retirada_acum,investido,% renda_acum
48,2014-01-31,2014,1,0.00,"5,000.00",0.00,0.00,0.00,0.00,"22,600.00",0.00,"22,600.00",0.00
49,2014-02-28,2014,2,0.00,0.00,0.00,0.00,0.00,0.00,"22,600.00",0.00,"22,600.00",0.00
50,2014-03-31,2014,3,0.00,0.00,0.00,0.00,0.00,0.00,"22,600.00",0.00,"22,600.00",0.00
51,2014-04-30,2014,4,0.00,885.17,0.00,0.00,0.00,0.00,"23,485.17",0.00,"23,485.17",0.00
52,2014-05-31,2014,5,0.00,0.00,0.00,0.00,0.00,0.00,"23,485.17",0.00,"23,485.17",0.00


In [5]:
df_stock_revenue = main_controller.stock_revenue_chart()

In [6]:
df_stock_revenue.head()

,Data,ano,mes,financeiro,aporte,retirada,rendimento,%,renda_acum,aporte_acum,retirada_acum,investido,% renda_acum
48,2014-01-31,2014,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
49,2014-02-28,2014,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50,2014-03-31,2014,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
51,2014-04-30,2014,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
52,2014-05-31,2014,5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [7]:
df1 = main_controller.acoes.resumo[main_controller.acoes.resumo['Data'] <= '2020-12-31'].groupby(['Papel', 'Data', 'ano', 'mes'])\
        .agg(financeiro=('Financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index()\
        .rename(columns={'Papel': 'Ativo'})
df1['Tipo'] = 'Ação'

df2 = main_controller.fiis.resumo[main_controller.fiis.resumo['Data'] <= '2020-12-31'].groupby(['Papel', 'Data', 'ano', 'mes'])\
            .agg(financeiro=('Financeiro', 'sum'), 
                aporte=('aporte', 'sum'), 
                retirada=('retirada', 'sum'), 
                rendimento=('rendimento', 'sum'))\
            .reset_index()\
            .rename(columns={'Papel': 'Ativo'})

df3 = main_controller.fi.resumo[main_controller.fi.resumo['data_posicao'] <= '2020-12-31'].groupby(['Nome', 'data_posicao', 'ano', 'mes'])\
            .agg(financeiro=('Total Bruto', 'sum'), 
                aporte=('aporte', 'sum'), 
                retirada=('retirada', 'sum'), 
                rendimento=('rendimento', 'sum'))\
            .reset_index()\
            .rename(columns={'data_posicao': 'Data', 
                            'Nome': 'Ativo'})      

df_graph1 = df1.append(df2, ignore_index=True).append(df3, ignore_index=True)
df_graph1 = df_graph1.groupby(['Ativo', 'Data', 'ano', 'mes'])\
        .agg(financeiro=('financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index().fillna(0)

df_graph1['%'] = df_graph1['rendimento'] / df_graph1['financeiro'] * 100
df_graph1['renda_acum'] = df_graph1['rendimento'].cumsum()
df_graph1['aporte_acum'] = df_graph1['aporte'].cumsum()
df_graph1['retirada_acum'] = df_graph1['retirada'].cumsum()
df_graph1['investido'] = df_graph1['aporte_acum'] - df_graph1['retirada_acum']
df_graph1['% renda_acum'] = df_graph1['rendimento'].cumsum() / (df_graph1['investido']) * 100  
df_graph1 = df_graph1.fillna(0)
df_graph1 = df_graph1[(df_graph1['Data'] >= '2014-01-01')]

df_graph1.head()

,Ativo,Data,ano,mes,financeiro,aporte,retirada,rendimento,%,renda_acum,aporte_acum,retirada_acum,investido,% renda_acum
48,AMZO34,2014-01-31,2014,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
49,AMZO34,2014-02-28,2014,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50,AMZO34,2014-03-31,2014,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
51,AMZO34,2014-04-30,2014,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
52,AMZO34,2014-05-31,2014,5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [8]:
df_graph1.groupby('Ativo').agg(total_aporte=('aporte', 'sum'), 
                              total_retirada=('retirada', 'sum')).reset_index()

,Ativo,total_aporte,total_retirada
0,AMZO34,"5,011.20",0.00
1,AZ Quest Multi FIC FIM,"15,000.00","18,591.84"
2,Azul Quantitativo FIM,0.00,"17,824.47"
3,BCFF11,"36,164.40",0.00
4,BNP Paribas Inflação FI RF,"10,300.00","10,168.14"
5,BPAC11,"7,190.00","15,500.00"
6,Bahia AM Maraú FIC de FIM,"73,000.00",0.00
7,CASH3,"3,030.00",0.00
8,CNES11,885.17,0.00
9,COGN3,"7,044.00",0.00


In [9]:

df1 = main_controller.acoes.resumo[main_controller.acoes.resumo['Data'] <= '2020-12-31']\
        .groupby(['Papel', 'Data', 'ano', 'mes'])\
        .agg(financeiro=('Financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index()\
        .rename(columns={'Papel': 'Nome'})
df1['Tipo'] = 'Ação'

df2 = main_controller.fiis.resumo[main_controller.fiis.resumo['Data'] <= '2020-12-31']\
        .groupby(['Papel', 'Data', 'ano', 'mes'])\
        .agg(financeiro=('Financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index()\
        .rename(columns={'Papel': 'Nome'})
df2['Tipo'] = 'FII'

df3 = main_controller.fi.resumo[main_controller.fi.resumo['data_posicao'] <= '2020-12-31']\
        .groupby(['Nome', 'data_posicao', 'ano', 'mes'])\
        .agg(financeiro=('Total Bruto', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index()\
        .rename(columns={'data_posicao': 'Data'})
df3['Tipo'] = 'FI'

df_graph1 = df1.append(df2, ignore_index=True).append(df3, ignore_index=True)

df_graph1 = df_graph1.groupby(['Tipo', 'Nome', 'Data', 'ano', 'mes'])\
        .agg(financeiro=('financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index().fillna(0)

df_graph1['%'] = df_graph1['rendimento'] / df_graph1['financeiro'] * 100
df_graph1['renda_acum'] = df_graph1['rendimento'].cumsum()
df_graph1['aporte_acum'] = df_graph1['aporte'].cumsum()
df_graph1['retirada_acum'] = df_graph1['retirada'].cumsum()
df_graph1['investido'] = df_graph1['aporte_acum'] - df_graph1['retirada_acum']
df_graph1['% renda_acum'] = df_graph1['rendimento'].cumsum() / (df_graph1['investido']) * 100  
df_graph1 = df_graph1.fillna(0)
df_graph1 = df_graph1[(df_graph1['Data'] >= '2014-01-01')]


df_graph2 = df_graph1.groupby(['Tipo', 'Nome'])\
                    .agg(rendimento=('rendimento', 'sum'),
                         aporte=('aporte', 'sum'), 
                         retirada=('retirada', 'sum')).reset_index()

df_graph2['rendimento'] = df_graph2['rendimento'].astype(np.int)

In [10]:
df_graph1.head()

,Tipo,Nome,Data,ano,mes,financeiro,aporte,retirada,rendimento,%,renda_acum,aporte_acum,retirada_acum,investido,% renda_acum
48,Ação,AMZO34,2014-01-31,2014,1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
49,Ação,AMZO34,2014-02-28,2014,2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50,Ação,AMZO34,2014-03-31,2014,3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
51,Ação,AMZO34,2014-04-30,2014,4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
52,Ação,AMZO34,2014-05-31,2014,5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [11]:
df_pie = df_graph1\
                    .groupby('Tipo')\
                    .agg(aporte=('aporte', 'sum'), 
                          retirada=('retirada', 'sum'))\
                    .reset_index()

df_pie['investido'] = df_pie['aporte'] - df_pie['retirada']

df_pie

,Tipo,aporte,retirada,investido
0,Ação,"122,099.50","39,950.00","82,149.50"
1,FI,"268,900.00","134,077.06","134,822.94"
2,FII,"65,871.73","5,199.00","60,672.73"


In [22]:
labels = df_pie['Tipo']
values = df_pie['investido']

# Use `hole` to create a donut-like pie chart
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3)])

fig.update_layout(
    showlegend=False, 
    margin=dict(l=20, r=20, t=20, b=20), 
    template='plotly_white', 
)

fig.show()

In [13]:
import plotly.express as px
df = px.data.tips()
fig = px.sunburst(data_frame=df_graph2, 
                  path=['Tipo', 'Nome'],
                  color='Tipo',
                  color_discrete_sequence=px.colors.qualitative.Pastel, 
                  maxdepth=-1)
fig.show()

In [14]:
fi_resumo = fi.resumo
acoes_resumo = acoes.resumo[acoes.resumo['Data'] <= '2020-12-31']
fiis_resumo = fiis.resumo[fiis.resumo['Data'] <= '2020-12-31']

NameError: name 'fi' is not defined

In [ ]:
display(fi_resumo.head())
display(acoes_resumo.head())
display(fiis_resumo.head())

**Mapeado todos os FIs**

Verifica se existe FI no extrato não mapeado

In [ ]:
#fundo_investimento.extrato[fundo_investimento.extrato['Nome'].str.contains('unknown')].head()
#extrato.aportes_fi_hist[extrato.aportes_fi_hist['Nome'] == 'unknown']

## Analise FIs

In [ ]:
resumo_fi = fundo_investimento.resumo

In [ ]:
print('Resumo por Fi:')

tmp = resumo_fi[resumo_fi['data_posicao'] <= '2020-12-31']\
        .groupby(['Nome'])\
        .agg(posicao=('Total Bruto', 'last'), 
             total_aporte=('aporte', 'sum'), 
             total_rendimento=('rendimento', 'sum'), 
             total_retirada=('retirada', 'sum')).reset_index()

tmp.sort_values(['total_rendimento'], ascending=False)

## Analise Ações

In [ ]:
resumo_acoes = acoes.calcula_resumo(2010, 2020)

In [ ]:
resumo_papel = resumo_acoes[resumo_acoes['Data'] <= '2020-12-31']\
        .groupby(['Papel'])\
        .agg(total_financeiro=('Financeiro', 'last'), 
            total_aporte=('aporte', 'sum'),
            total_retirada=('retirada', 'sum'),
            total_rendimento=('rendimento', 'sum')).reset_index()

resumo_papel['%'] = resumo_papel['total_rendimento'] / resumo_papel['total_aporte'] * 100

resumo_papel.sort_values('total_rendimento', ascending=False)

## FIIs

In [ ]:
resumo_fiis = fiis.calcula_resumo(2010, 2020)

In [ ]:
resfiis_papel = resumo_fiis[resumo_fiis['Data'] <= '2020-12-31']\
        .groupby(['Papel'])\
        .agg(total_financeiro=('Financeiro', 'last'), 
            total_aporte=('aporte', 'sum'),
            total_retirada=('retirada', 'sum'),
            total_rendimento=('rendimento', 'sum')).reset_index()

resfiis_papel['%'] = resfiis_papel['total_rendimento'] / resfiis_papel['total_aporte'] * 100

resfiis_papel.sort_values('total_rendimento', ascending=False)

## Grafico de rendimento

In [ ]:
df1 = resumo_acoes[resumo_acoes['Data'] <= '2020-12-31'].groupby(['Data', 'ano', 'mes'])\
        .agg(financeiro=('Financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index()

In [ ]:
df2 = resumo_fiis[resumo_fiis['Data'] <= '2020-12-31'].groupby(['Data', 'ano', 'mes'])\
        .agg(financeiro=('Financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index()

In [ ]:
df3 = resumo_fi[resumo_fi['data_posicao'] <= '2020-12-31'].groupby(['data_posicao', 'ano', 'mes'])\
        .agg(financeiro=('Total Bruto', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index()\
        .rename(columns={'data_posicao': 'Data'})

In [ ]:
df_graph1 = df1.append(df2, ignore_index=True).append(df3, ignore_index=True)

df_graph1 = df_graph1.groupby(['Data', 'ano', 'mes'])\
        .agg(financeiro=('financeiro', 'sum'), 
            aporte=('aporte', 'sum'), 
            retirada=('retirada', 'sum'), 
            rendimento=('rendimento', 'sum'))\
        .reset_index().fillna(0)

df_graph1['%'] = df_graph1['rendimento'] / df_graph1['financeiro'] * 100
df_graph1['renda_acum'] = df_graph1['rendimento'].cumsum()
df_graph1['aporte_acum'] = df_graph1['aporte'].cumsum()
df_graph1['retirada_acum'] = df_graph1['retirada'].cumsum()
df_graph1['investido'] = df_graph1['aporte_acum'] - df_graph1['retirada_acum']
df_graph1['% renda_acum'] = df_graph1['rendimento'].cumsum() / (df_graph1['investido']) * 100

In [ ]:
df_graph1.fillna(0)

In [ ]:
df_graph1.tail(1)

In [ ]:
def create_graph1(df):

    fig = go.Figure()
    data = []

    fig.add_trace(
        go.Scatter(x=df['Data'],
                y=df['%'],
                mode='lines',
                name='% Rendimento',
                textposition='top center',
                text=df['%'].apply(lambda x: f'{x:,.2f}%'),
                marker=dict(size=7),
                line=dict(color='rgb(115,115,115)', width=1.8),
                opacity=.8))  

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    fig.update_layout(
        template='plotly_white', 
        legend_orientation='h', 
        #height=600, 
        #width=1000, 
        title={
            'y':0.9,
            'xanchor': 'left',
            'yanchor': 'top'}
        
    )


    fig.show()

In [ ]:
create_graph1(df_graph1[(df_graph1['Data'] >= '2014-01-01')].sort_values('Data'))

In [ ]:
def create_graph3(df):

    fig = go.Figure()
    data = []

    fig.add_trace(
        go.Scatter(x=df['Data'],
                y=df['% renda_acum'],
                mode='lines',
                name='% Rendimento',
                textposition='top center',
                text=df['% renda_acum'].apply(lambda x: f'{x:,.2f}%'),
                marker=dict(size=7),
                line=dict(color='rgb(115,115,115)', width=1.8),
                opacity=.8))

    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    fig.update_layout(
        template='plotly_white', 
        legend_orientation='h', 
        #height=600, 
        #width=1000, 
        title={
            'y':0.9,
            'xanchor': 'left',
            'yanchor': 'top'}
        
    )


    fig.show()
    
create_graph3(df_graph1[df_graph1['Data'] >= '2014-01-01'].sort_values('Data'))

In [ ]:
def create_graph2(df):

    fig = go.Figure()
    data = []

    fig.add_trace(
        go.Scatter(x=df['Data'],
            y=df['renda_acum'],
            mode='lines',
            name = 'Rendimento Acumulado',
            text=df['renda_acum'].apply(lambda x: f'{x/1000:,.0f}K'),
            #textposition='outside',
            line=dict(color='rgb(115,115,115)'), 
            line_shape='linear'
            ))            
    
    fig.update_xaxes(
        rangeslider_visible=True,
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    )

    fig.update_layout(
        template='plotly_white', 
        legend_orientation='h', 
        #height=600, 
        #width=1000, 
        title={
            'y':0.9,
            'xanchor': 'left',
            'yanchor': 'top'}
        
    )


    fig.show()

In [ ]:
create_graph2(df_graph1[df_graph1['Data'] >= '2014-01-01'].sort_values('Data'))

In [ ]:
import plotly.express as px

df_tmp = df_graph1[df_graph1['Data'] >= '2014-01-01'].sort_values('Data')

fig = px.line(df_tmp, x='Data', y='renda_acum', title='Time Series with Range Slider and Selectors')

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()